In [13]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ce40e9b043decde533bad6a9c9cda7864e3cbfa12fe6b76ba30c012f608775e7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
!pip install fuzzywuzzy

In [151]:
import pandas as pd
relation_df = pd.read_csv("/content/relation_extraction_rebel.csv",index_col=0)
relation_df.head()

,Subject,Relation,Object
0,action,part of,strategy
1,risk order,studied by,risk management
2,risk response,studied by,risk management
3,pmi,instance of,standard
4,comparable situation,subclass of,historical review


In [152]:
import pandas as pd
df = pd.read_csv("/content/PMI_PMBOK_Definitions.csv")
df.head()

,Concept,Cleaned_Description,References,Sentences,KeyWords_KeyBert,Synonyms_SBert,Description,Definition
0,PROJECT RISK MANAGEMENT,project risk management includes process condu...,['https://i.ibb.co/pWPHv3r/396.png'],['project risk management includes process con...,"['project risk managementall', 'practice proje...","['project risk managementall', 'practice proje...",Project Risk Management includes the processes...,Project Risk Management includes the processe...
1,Plan Risk Management,plan risk management process defining conduct ...,['https://i.ibb.co/rssfq3x/401.jpg'],['plan risk management process defining conduc...,"['risk management process requires', 'project ...","['plan risk management process', 'plan risk ma...",Plan Risk Management is the process of definin...,Plan Risk Management is the process of defini...
2,Plan Risk Management Inputs,11.1.1.1 project charter described section 4.1...,"['https://i.ibb.co/fXjdfx5/402.jpg', '4.1.3.1'...",['11.1.1.1 project charter described section 4...,"['responsibility managing risk project', 'risk...",[],11.1.1.1 PROJECT CHARTER Described in Section ...,The project charter documents the high-level ...
3,Plan Risk Management Tools and Techniques,11.1.2.1 expert judgment described section 4.1...,"['4.1.2.1', '13.1.2.3', '4.1.2.1', '13.1.2.3']",['11.1.2.1 expert judgment described section 4...,"['risk management specific need', 'conducting ...",['risk management plan developed'],11.1.2.1 EXPERT JUDGMENT Described in Section...,Expertise should be considered from individua...
4,Plan Risk Management Outputs,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...",['11.1.3.1 risk management planthe risk manage...,"['required project risk management', 'project ...",[],11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,The risk management plan is a component of th...


In [153]:
def extract_from_first_word(text):
    words = text.split()
    substring = ' '.join(words[words.index(next(filter(str.strip, words))):])
    return substring

# Apply the function to the DataFrame column
df['Concept'] = df['Concept'].apply(lambda x: extract_from_first_word(x))

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embedding for both lists
embeddings1 = model.encode(relation_df["Object"].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df["Concept"].tolist(), convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# For each 'Subject' in relation_df, find the index of the maximum similarity score
closest_concepts = []
for i in range(len(relation_df["Object"])):
    max_index = int(np.argmax(cosine_scores[i]))
    closest_concepts.append(df["Concept"].iloc[max_index])

# Update 'Subject' with the closest 'Concept'
relation_df['Object'] = closest_concepts


In [162]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
a= 0
b=0
l=[]
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embedding for both lists
embeddings1 = model.encode(relation_df["Subject"].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df["Concept"].tolist(), convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# For each 'Object' in relation_df, find the index of the maximum similarity score
closest_concepts = []
for i in range(len(relation_df["Subject"])):
    max_index = int(np.argmax(cosine_scores[i]))
    if cosine_scores[i][max_index] > 0.75:
        a += 1
        closest_concepts.append(df["Concept"].iloc[max_index])
    else:
        b += 1
        l.append(relation_df["Subject"].iloc[i])
        closest_concepts.append(relation_df["Subject"].iloc[i])  # Keep the original value if score is below 0.75

# Update 'Object' with the closest 'Concept' only if the score is above 0.75
relation_df['Subject'] = closest_concepts
print(a,b)

418 1075


In [163]:
relation_df.head()

,Subject,Relation,Object,Object_Definition,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References
0,action,part of,strategy,[{'summary_text': ' Risk response planning sho...,NaN,NaN,NaN
1,risk order,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
2,Risk Response Strategies,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
3,pmi,instance of,standard,[{'summary_text': ' A PMI practice standard pr...,NaN,NaN,NaN
4,comparable situation,subclass of,Historical Review,Historical reviews are based on what occurred...,"['project organization comparable project', 'b...",[],[]


In [156]:
concepts_list = df["Concept"].tolist()

In [164]:
a=[]
for i, row in relation_df.iterrows():
    if row["Subject"] not in concepts_list:

        a.append(row["Subject"])
print(len(a))

1075


In [165]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embedding for both lists
embeddings1 = model.encode(a, convert_to_tensor=True)
embeddings2 = model.encode(df["Concept"].tolist(), convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)
d={}
# For each 'Subject' in relation_df, find the index of the maximum similarity score
closest_concepts = []
for i in range(len(a)):
    max_index = int(np.argmax(cosine_scores[i]))
    d[a[i]] = df["Definition"].iloc[max_index]
    #closest_concepts.append(df["Definition"].iloc[max_index])

# Update 'Subject' with the closest 'Concept'
#relation_df['Object_def'] = closest_concepts
print(d)

{'action': "[{'summary_text': ' Project planning tools are used to turn the chosen strategies into concrete actions and to integrate these into existing plans . The corresponding actions may be unconditional i.e., integrated into the project management plan or contingent on a trigger condition and predefined as a contingency response strategy . Project planners use tools such as project planning tools to turn strategy into concrete action plans .'}]", 'risk order': "[{'summary_text': ' The selection and prioritization of risks must be linked to the project objectives . The risk management plan will define the relative importance to be assigned to these objectives e.g. reliability takes precedence over time, etc. This may be in the form of numerical weights that can be applied to create a single weighted sum over all objectives for each risk .'}]", 'pmi': "[{'summary_text': ' A PMI practice standard provides guidelines on the mechanics e.g., basics, fundamentals, step-by-step usage guid

In [166]:
for key, value in d.items():
    if key in relation_df["Subject"].values:
        relation_df.loc[relation_df["Subject"] == key, "Subject_Definition"] = value

In [167]:
for idx, subject in relation_df["Subject"].items():
    if subject in df["Concept"].values:
        concept_index = df.index[df["Concept"] == subject].tolist()[0]
        for col in ["Definition", "KeyWords_KeyBert", "Synonyms_SBert", "References"]:
            relation_df.at[idx, f"Subject_{col}"] = df.at[concept_index, col]


In [168]:
relation_df.head()

,Subject,Relation,Object,Object_Definition,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References,Subject_Definition,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References
0,action,part of,strategy,[{'summary_text': ' Risk response planning sho...,NaN,NaN,NaN,[{'summary_text': ' Project planning tools are...,NaN,NaN,NaN
1,risk order,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[],[{'summary_text': ' The selection and prioriti...,NaN,NaN,NaN
2,Risk Response Strategies,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[],[{'summary_text': ' The project manager should...,"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[]
3,pmi,instance of,standard,[{'summary_text': ' A PMI practice standard pr...,NaN,NaN,NaN,[{'summary_text': ' A PMI practice standard pr...,NaN,NaN,NaN
4,comparable situation,subclass of,Historical Review,Historical reviews are based on what occurred...,"['project organization comparable project', 'b...",[],[],Scenario analysis for risk response planning ...,NaN,NaN,NaN


In [169]:
count_equal_subject_object = len(relation_df[relation_df["Subject"] == relation_df["Object"]])
print("Number of occurrences where Subject is equal to Object:", count_equal_subject_object)


Number of occurrences where Subject is equal to Object: 71


In [170]:
relation_df = relation_df[relation_df["Subject"] != relation_df["Object"]]

In [171]:
relation_df.head(20)

,Subject,Relation,Object,Object_Definition,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References,Subject_Definition,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References
0,action,part of,strategy,[{'summary_text': ' Risk response planning sho...,NaN,NaN,NaN,[{'summary_text': ' Project planning tools are...,NaN,NaN,NaN
1,risk order,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[],[{'summary_text': ' The selection and prioriti...,NaN,NaN,NaN
2,Risk Response Strategies,studied by,Good Risk Management Practice,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[],[{'summary_text': ' The project manager should...,"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[]
3,pmi,instance of,standard,[{'summary_text': ' A PMI practice standard pr...,NaN,NaN,NaN,[{'summary_text': ' A PMI practice standard pr...,NaN,NaN,NaN
4,comparable situation,subclass of,Historical Review,Historical reviews are based on what occurred...,"['project organization comparable project', 'b...",[],[],Scenario analysis for risk response planning ...,NaN,NaN,NaN
5,project uncertain,facet of,project,An appropriate model of the project should be...,NaN,NaN,NaN,An appropriate model of the project should be...,NaN,NaN,NaN
6,waste,subclass of,resource,Project risk analysis management PRAM guide s...,NaN,NaN,NaN,The principles of Project Risk Management des...,NaN,NaN,NaN
7,status change risk characteristic,facet of,Identify Risks,Identify Risks is the process of identifying ...,"['risks process performed project', 'project r...",[],"['https://i.ibb.co/47Jwqzc/409.jpg', 'https://...",[{'summary_text': ' In order for all risks to ...,NaN,NaN,NaN
8,Categorize Risk Causes,subclass of,framework,Project Management Institute PMI practice sta...,NaN,NaN,NaN,Categorizing risks appropriately may lead to ...,"['risk project identifying', 'project identify...",[],[]
9,project organization,subclass of,organization,The feasibility of risk management planning i...,NaN,NaN,NaN,An appropriate model of the project should be...,NaN,NaN,NaN


In [172]:
relation_df.to_csv("relation_df2.csv",index=False)

In [173]:
relation_df2 = relation_df.copy()

In [ ]:
for idx, subject in relation_df2["Subject"].items():
    if subject in df["Concept"].values:
        concept_index = df.index[df["Concept"] == subject].tolist()[0]
        for col in ["Description", "Definition", "KeyWords_KeyBert", "Synonyms_SBert", "References"]:
            relation_df2.at[idx, f"Subject_{col}"] = df.at[concept_index, col]


In [32]:
for idx, subject in relation_df2["Object"].items():
    if subject in df["Concept"].values:
        concept_index = df.index[df["Concept"] == subject].tolist()[0]
        for col in ["Description", "Definition", "KeyWords_KeyBert", "Synonyms_SBert", "References"]:
            relation_df2.at[idx, f"Object_{col}"] = df.at[concept_index, col]


In [33]:
relation_df2.head()

,Subject,Relation,Object,Subject_Description,Subject_Definition,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References,Object_Description,Object_Definition,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References
0,Action Planning,part of,Develop Strategies before Tactical Responses,Project planning tools are used to turn the ch...,[{'summary_text': ' Project planning tools are...,"['project planning', 'planning tool used', 'pr...",[],[],Risk response planning should be carried out i...,[{'summary_text': ' Risk response planning sho...,"['response planning carried', 'response planni...",[],[]
1,Risk Prioritization and Selection Guidelines,studied by,Good Risk Management Practice,The selection and prioritization of risks mus...,[{'summary_text': ' The selection and prioriti...,"['project objectives risk management', 'priori...","['prioritization risk', 'selection prioritizat...",[],Project Risk Management is a valuable componen...,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
2,Risk Response Strategies,studied by,Good Risk Management Practice,The project manager should develop risk respon...,[{'summary_text': ' The project manager should...,"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[],Project Risk Management is a valuable componen...,Project Risk Management is a valuable compone...,"['project risk management general', 'project r...",[],[]
4,Scenario Analysis,subclass of,Historical Review,Scenario analysis for risk response planning ...,Scenario analysis for risk response planning ...,"['analysis risk response planning', 'scenario ...","['scenario analysis', 'scenario analysis risk']",[],Historical reviews are based on what occurred ...,Historical reviews are based on what occurred...,"['project organization comparable project', 'b...",[],[]
6,Conclusion,subclass of,REFERENCES,The principles of Project Risk Management desc...,The principles of Project Risk Management des...,"['project risk management provides', 'project ...",[],[],Association for Project Management. 2004. Proj...,Project risk analysis management PRAM guide s...,"['2004 project risk analysis', 'management 200...",[],[]


In [174]:
import ast
l = []
for desc in relation_df2["Subject_Definition"]:
    try:
        for j in ast.literal_eval(desc):
            l.append(j['summary_text'])
    except Exception as e:
        l.append(desc)
relation_df2.drop("Subject_Definition",inplace=True,axis=1)
relation_df2["Subject_Definition"] = l

In [175]:
import ast
l = []
for desc in relation_df2["Object_Definition"]:
    try:
        for j in ast.literal_eval(desc):
            l.append(j['summary_text'])
    except Exception as e:
        l.append(desc)
relation_df2.drop("Object_Definition",inplace=True,axis=1)
relation_df2["Object_Definition"] = l

In [176]:
relation_df2.head()

,Subject,Relation,Object,Object_KeyWords_KeyBert,Object_Synonyms_SBert,Object_References,Subject_KeyWords_KeyBert,Subject_Synonyms_SBert,Subject_References,Subject_Definition,Object_Definition
0,action,part of,strategy,NaN,NaN,NaN,NaN,NaN,NaN,Project planning tools are used to turn the c...,Risk response planning should be carried out ...
1,risk order,studied by,Good Risk Management Practice,"['project risk management general', 'project r...",[],[],NaN,NaN,NaN,The selection and prioritization of risks mus...,Project Risk Management is a valuable compone...
2,Risk Response Strategies,studied by,Good Risk Management Practice,"['project risk management general', 'project r...",[],[],"['project level risks', 'develop risk response...","['develop risk response strategy', 'risk respo...",[],The project manager should develop risk respo...,Project Risk Management is a valuable compone...
3,pmi,instance of,standard,NaN,NaN,NaN,NaN,NaN,NaN,A PMI practice standard provides guidelines o...,A PMI practice standard provides guidelines o...
4,comparable situation,subclass of,Historical Review,"['project organization comparable project', 'b...",[],[],NaN,NaN,NaN,Scenario analysis for risk response planning ...,Historical reviews are based on what occurred...


In [178]:
relation_df2.to_csv("FINAL_DATASET2.csv",index=False)